#1


In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import os

# Charger le jeu de données CNN/DailyMail
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Fonction de nettoyage du texte
def clean_text(text):
    # Supprimer les caractères spéciaux et normaliser le texte
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Supprimer les caractères spéciaux
    text = re.sub(r'\s+', ' ', text).strip()  # Supprimer les espaces multiples
    return text

# Nettoyage des articles et résumés
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].map(lambda x: {
        "article": clean_text(x["article"]),
        "highlights": clean_text(x["highlights"])
    })

# Préparation des données pour la division
data = pd.DataFrame(dataset["train"])
articles = data["article"]
summaries = data["highlights"]

# Division en ensembles (80/10/10)
X_train, X_temp, y_train, y_temp = train_test_split(articles, summaries, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Création des ensembles finaux
train_data = pd.DataFrame({"article": X_train, "summary": y_train})
val_data = pd.DataFrame({"article": X_val, "summary": y_val})
test_data = pd.DataFrame({"article": X_test, "summary": y_test})

# Create the directory if it doesn't exist
os.makedirs("/mnt/data", exist_ok=True)

# Sauvegarder les ensembles nettoyés
train_data_path = "/mnt/data/cnn_dailymail_train.csv"
val_data_path = "/mnt/data/cnn_dailymail_val.csv"
test_data_path = "/mnt/data/cnn_dailymail_test.csv"

train_data.to_csv(train_data_path, index=False)
val_data.to_csv(val_data_path, index=False)
test_data.to_csv(test_data_path, index=False)

train_data_path, val_data_path, test_data_path

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

('/mnt/data/cnn_dailymail_train.csv',
 '/mnt/data/cnn_dailymail_val.csv',
 '/mnt/data/cnn_dailymail_test.csv')

#2

In [3]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=333bfbd236e97561b4aaab1ab903211da8bd2be637ca8aa79586e82a74c3a793
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [6]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
import evaluate
import numpy as np

# Charger le modèle et le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Diviser l'article en phrases
sentences = sent_tokenize(articles[0])

# Encoder les phrases avec BERT
def encode_sentences(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        # Utiliser la représentation [CLS] comme embedding de la phrase
        cls_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
        embeddings.append(cls_embedding.squeeze())
    return np.array(embeddings)

sentence_embeddings = encode_sentences(sentences)

# Calculer l'embedding du document entier (moyenne des phrases)
document_embedding = np.mean(sentence_embeddings, axis=0, keepdims=True)

# Calculer la similarité cosinus entre chaque phrase et le document
similarities = cosine_similarity(sentence_embeddings, document_embedding).squeeze()

# Sélectionner les phrases avec les scores les plus élevés
n_selected_sentences = 2  # Nombre de phrases à inclure dans le résumé
selected_indices = np.argsort(similarities)[-n_selected_sentences:]
selected_indices = sorted(selected_indices)  # Conserver l'ordre original des phrases

# Construire le résumé extractif
extracted_summary = " ".join([sentences[i] for i in selected_indices])

# Évaluer avec BLEU et ROUGE
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

rouge_score = rouge.compute(predictions=[extracted_summary], references=[summaries[0]])
bleu_score = bleu.compute(predictions=[extracted_summary], references=[summaries[0]])
extracted_summary, rouge_score, bleu_score


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

('LONDON England Reuters Harry Potter star Daniel Radcliffe gains access to a reported 20 million 411 million fortune as he turns 18 on Monday but he insists the money wont cast a spell on him Daniel Radcliffe as Harry Potter in Harry Potter and the Order of the Phoenix To the disappointment of gossip columnists around the world the young actor says he has no plans to fritter his cash away on fast cars drink and celebrity parties I dont plan to be one of those people who as soon as they turn 18 suddenly buy themselves a massive sports car collection or something similar he told an Australian interviewer earlier this month I dont think Ill be particularly extravagant The things I like buying are things that cost about 10 pounds books and CDs and DVDs At 18 Radcliffe will be able to gamble in a casino buy a drink in a pub or see the horror film Hostel Part II currently six places below his number one movie on the UK box office chart Details of how hell mark his landmark birthday are unde

**ROUGE-1 (14.75%)** :Indique qu'environ 15% des mots du résumé généré sont également présents dans le résumé de référence. Ce score est relativement faible, ce qui peut indiquer que le résumé généré ne capte pas bien les mots-clés. **ROUGE-2 (11.93%)** :montre que la cohérence contextuelle pourrait être améliorée. **ROUGE-L (14.75%)** : Basé sur les plus longues sous-séquences communes. il indique que le résumé généré suit peu la structure du résumé de référence.

**BLEU (5.49%)** :un score est très faible, suggérant que le résumé généré diffère significativement du résumé de référence en termes de n-grammes. Egalement, la précision diminue fortement à mesure que la longueur des n-grammes augmente. Cela reflète une perte de fidélité contextuelle et de structure grammaticale. Enfin,le ratio de longueur entre le résumé généré et le résumé de référence est de 11.84, indiquant que le résumé généré est bien trop long. Ceci affecte directement les scores BLEU, car les n-grammes inutiles diminuent la précision globale

In [7]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Charger le modèle et tokenizer BART
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [8]:
print(type(train_data['article']))
print(type(train_data['summary']))


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [9]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
from torch.utils.data import DataLoader

# Charger le modèle et tokenizer BART
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Fonction pour prétraiter les données
def preprocess_data(articles, summaries, tokenizer, max_length=512):
    # Vérifier si les entrées sont des pandas.Series et les convertir en listes
    if isinstance(articles, pd.Series):
        articles = articles.tolist()
    if isinstance(summaries, pd.Series):
        summaries = summaries.tolist()

    # Tokenizer les articles et résumés
    inputs = tokenizer(articles, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    targets = tokenizer(summaries, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    return inputs, targets

# Conversion des colonnes en listes
train_articles = train_data['article'].tolist()
train_summaries = train_data['summary'].tolist()
val_articles = val_data['article'].tolist()
val_summaries = val_data['summary'].tolist()

# Préparer les données pour DataLoader
train_inputs, train_targets = preprocess_data(train_articles, train_summaries, bart_tokenizer)
val_inputs, val_targets = preprocess_data(val_articles, val_summaries, bart_tokenizer)

# Créer des DataLoaders pour l'entraînement et la validation
train_dataset = torch.utils.data.TensorDataset(train_inputs["input_ids"], train_targets["input_ids"])
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

val_dataset = torch.utils.data.TensorDataset(val_inputs["input_ids"], val_targets["input_ids"])
val_loader = DataLoader(val_dataset, batch_size=8)

In [10]:
test_articles=test_data['article'].tolist()
test_summaries=test_data['summary'].tolist()

In [11]:
test_articles[0]

'CNN Jenni Rivera Add the MexicanAmerican superstars name to a long list of beloved celebrities killed in the crash of a charter aircraft That list includes singeractress Aaliyah golfer Payne Stewart and musicians Ronnie Van Zant Buddy Holly and Ritchie Valens In 2008 Travis Barker of Blink182 and Adam DJ AM Goldstein were critically injured and four other passengers were killed when their charter jet crashed on takeoff in South Carolina Overall the industry is overwhelmingly safe Yet Sundays crash in northern Mexico that killed Rivera and six others including a pilot and copilot has focused attention on charter safety Related Plane owner linked to troubled businessman One red flag experts say is the 43yearold planes age The Learjet is a pretty reliable airplane but this one was really old says Ladd Sanger an attorney and licensed pilot who specializes in plane crashes As a general rule over 40 years old and I think you need to be concerned However on its own age is not an indicator of

In [12]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
import evaluate

# Charger le modèle et le tokenizer BART
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Charger les métriques ROUGE et BLEU
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# Fonction pour générer un résumé avec BART
def generate_summary(text, model, tokenizer, max_length=150):
    model.eval()  # Mettre le modèle en mode évaluation
    with torch.no_grad():
        # Tokeniser l'article
        inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding=True)
        input_ids = inputs["input_ids"].to(model.device)

        # Générer le résumé avec le modèle
        summary_ids = model.generate(input_ids, max_length=max_length, num_beams=4, length_penalty=2.0, early_stopping=True)

        # Décoder le résumé généré
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

# Exemple de texte à résumer
article = test_articles[0]

# Générer le résumé avec BART
generated_summary = generate_summary(article, bart_model, bart_tokenizer)

# Exemple de résumé de référence pour l'évaluation
reference_summary = test_summaries[0]

# Évaluer le résumé généré avec ROUGE
rouge_score = rouge.compute(predictions=[generated_summary], references=[reference_summary])

# Évaluer le résumé généré avec BLEU
bleu_score = bleu.compute(predictions=[generated_summary], references=[reference_summary])

# Afficher les résultats
generated_summary, rouge_score, bleu_score


('Jenni Rivera and six others were killed in a charter jet crash in northern Mexico on Sunday. The crash has focused attention on charter safety. Experts say travelers can reduce their risk of choosing a charter thats poorly run or even illegal by asking a few smart questions. Make sure the charter is well established and employs its own pilots.',
 {'rouge1': 0.32183908045977017,
  'rouge2': 0.09411764705882353,
  'rougeL': 0.2528735632183908,
  'rougeLsum': 0.2528735632183908},
 {'bleu': 0.0,
  'precisions': [0.171875, 0.047619047619047616, 0.016129032258064516, 0.0],
  'brevity_penalty': 1.0,
  'length_ratio': 2.3703703703703702,
  'translation_length': 64,
  'reference_length': 27})

ROUGE-1 (0.3218) : Ce score montre un chevauchement modéré (32 % des unigrams correspondent).
ROUGE-2 (0.0941) : Le score est faible (9 %), ce qui suggère que la structure phrastique de la sortie diffère de celle de la référence.
ROUGE-L (0.2529) : Avec un score de 25 %, la cohérence structurale est limitée.

BLEU (0.0) : Le score global est nul, car les n-grams plus longs (3-grams, 4-grams) n'ont aucun chevauchement avec la référence.
1-gram : 17.18 % des mots sont corrects.
2-gram : Seulement 4.76 % de chevauchement, ce qui montre que la continuité des mots est faible.
3-gram et 4-gram : Absence totale de correspondance.
Longueur et pénalité de brièveté : La sortie est beaucoup plus longue que la référence (ratio 2.37). La sortie n'est pas plus courte, aucune pénalité n'est appliquée.

Les scores ROUGE indiquent qu’il y a une certaine pertinence unitaire (1-gram) mais que les structures plus complexes (2-gram) sont mal reproduites.
BLEU montre une faible précision et un manque de fluidité. Les n-grams plus longs (au-delà de 2 mots) ne correspondent pas, indiquant un problème de cohérence.

#4

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bart_model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [15]:
def generate_summaries(test_data, model, tokenizer, max_length=150):
    model.eval()  # Mettre le modèle en mode évaluation
    summaries = []
    with torch.no_grad():
        for article in test_data:
            # Tokenisation de l'article
            inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True, padding=True)
            input_ids = inputs["input_ids"].to(model.device)  # Déplacer les entrées sur le même appareil que le modèle

            # Générer le résumé avec le modèle
            summary_ids = model.generate(
                input_ids,
                max_length=max_length,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True
            )

            # Décoder les IDs des tokens en texte
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)
    return summaries


In [ ]:
test_summaries = generate_summaries(test_data['article'], bart_model, bart_tokenizer)


In [ ]:
from datasets import load_metric

# Charger les métriques
rouge_metric = load_metric("rouge")
bleu_metric = load_metric("bleu")

def compute_metrics(predictions, references):
    # Calcul des scores ROUGE
    rouge_scores = rouge_metric.compute(predictions=predictions, references=references)

    # Calcul des scores BLEU
    bleu_scores = bleu_metric.compute(predictions=predictions, references=references)

    return rouge_scores, bleu_scores

# Comparer les résumés générés (abstraits et extractifs) avec les résumés de référence
rouge_scores, bleu_scores = compute_metrics(test_summaries, test_data['summary'])

print("ROUGE Scores:", rouge_scores)
print("BLEU Scores:", bleu_scores)


#5

In [ ]:
# Exemple d'application du modèle sur un autre texte
external_article = "Texte provenant d'une autre source"
generated_summary_external = generate_summary(external_article, bart_model, bart_tokenizer)

# Comparaison avec un résumé de référence pour l'évaluation
reference_summary_external = "Résumé de référence pour l'article externe"
rouge_score_external = rouge.compute(predictions=[generated_summary_external], references=[reference_summary_external])
bleu_score_external = bleu.compute(predictions=[generated_summary_external], references=[reference_summary_external])

generated_summary_external, rouge_score_external, bleu_score_external


Variantes du modèle :
Pour expérimenter avec différentes configurations et architectures, vous pouvez essayer des différents modèles ou ajuster certains paramètres du modèle BART.

un. Testeur différents modèles pré-entraînés (par exemple GPT) :
Vous pouvez tester des modèles pré-entraînés comme GPT (Generative Pretrained Transformer), T5 (Text-to-Text Transfer Transformer) ou même des versions spécifiques de BART pour voir comment chaque modèle se comporte sur les tâches de résumé.
Voici un exemple avec GPT :

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Charger GPT2
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

# Fonction de génération de texte avec GPT2
def generate_gpt2_summary(text, model, tokenizer, max_length=150):
    model.eval()  # Mettre le modèle en mode évaluation
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding=True)
        input_ids = inputs["input_ids"].to(model.device)
        summary_ids = model.generate(input_ids, max_length=max_length, num_beams=4, length_penalty=2.0, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

# Exemple d'application sur un texte
generated_summary_gpt2 = generate_gpt2_summary(external_article, gpt2_model, gpt2_tokenizer)
print(generated_summary_gpt2)


In [ ]:
# Exemple d'entraînement avec différents nombres d'époques et tailles de batchs
from transformers import AdamW

# Paramètres d'entraînement
epochs = 5
batch_size = 16
optimizer = AdamW(bart_model.parameters(), lr=5e-5)

# Exemple d'entraînement avec une nouvelle configuration
for epoch in range(epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, target_ids = batch
        outputs = bart_model(input_ids=input_ids, labels=target_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")


3. Exploration de techniques avancées :
Les techniques avancées peuvent aider à améliorer les performances en particulier dans les scénarios de texte complexe ou de longue longueur. Voici quelques idées :

un. Régularisation (Décrochage) :
Le dropout est une technique de régularisation qui consiste à éteindre aléatoirement certaines connexions neuronales pendant l'entraînement. Cela permet de rendre le modèle moins susceptible au surajustement.

In [ ]:
from transformers import BartForConditionalGeneration

# Application de dropout
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn", config={"dropout": 0.2})


In [ ]:
# Ajuster la longueur du résumé
summary_ids = bart_model.generate(input_ids, max_length=100, min_length=50, length_penalty=1.2)
